### Test DDF against LSST AGN SC DDF Cadence White Paper

In [1]:
#Please enter your SciServer username between the single quotes below!
your_username = ''

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

Import the sims_maf modules needed.

In [3]:
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

### **Important:**  
The following code is needed before importing the `opsimUtils` module if the module is not in the same directory as the current notebook. That is, you need add the directory where the `opsimUtils` module is located to the Python search path. 

In [4]:
# add opsimUtils module path to search
import sys
sys.path.insert(0, '../Scripts_NBs/')

In [5]:
# import convenience functions
from opsimUtils import *

#### 1. Build connections to the OpSims databases
The first step is to initiate opsim database objects and result database objects for the opsim databases that you want to run metrics on. Two paths are needed here:
1. `dbDir`: The path to the OpSim database directory
2. `outDir`: The path to the directory where you want to save the metric metadata.

By providing these two paths, the function `connect_dbs()` will initiate connections and store the metadata to the designated directory.

In [6]:
if your_username == '': # do NOT put your username here, put it in the cell at the top of the notebook.
    raise Exception('Please provide your username!  See the top of the notebook.')

dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.4/'
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/DDF/'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [7]:
# two dictionary are returned by the following function, 
# One (opSimDbs) is a dictionary storing all database objects
# Another (resultDbs) is a dictionary consist of the objects directing MAF where to save metric metadata
# Both dictionaries are indexed by OpSim run names
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

In [8]:
dbRuns = show_opsims(dbDir)
dbRuns[0:5] # only show first 5 opsims

['wfd_depth_scale0.85_noddf_v1.4_10yrs',
 'twilight_neo_mod2_v1.4_10yrs',
 'weather_0.3_v1.4_10yrs',
 'pair_strategy_4_v1.4_10yrs',
 'short_exp_2ns_1expt_v1.4_10yrs']

#### 2. Declare Coadd metrics to run on above OpSims

In [10]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']
propIds = [2, 3, 5, 6]
coaddBDict = {}

In [11]:
metricNameTmp = 'coadd_{}_{}'
coaddSlicer = slicers.HealpixSlicer(nside=64)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [12]:
for propId in propIds:
    for band in bands:
        metricName = metricNameTmp.format(propId, band)
        coaddMetric = metrics.Coaddm5Metric(metricName=metricName)
        coaddConstraint = 'filter = "{}"'.format(band)
        coaddConstraint += ' and proposalId = {}'.format(propId)
        coaddBDict[metricName] = metricBundles.MetricBundle(coaddMetric, \
                                coaddSlicer, coaddConstraint)

In [13]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
for metric in coaddBDict:
    coaddBDict[metric].setSummaryMetrics(summaryMetrics)

#### 3. Declare NVisits metrics to run on above OpSims

In [24]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']
propIds = [2, 3, 5, 6]
nvisitBDict = {}

In [31]:
metricNameTmp = 'nvisit_{}_{}'
nvisitSlicer = slicers.HealpixSlicer(nside=32)

Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes


In [26]:
for propId in propIds:
    for band in bands:
        metricName = metricNameTmp.format(propId, band)
        nvisitMetric = metrics.CountMetric('observationStartMJD', metricName=metricName)
        nvisitConstraint = 'filter = "{}"'.format(band)
        nvisitConstraint += ' and proposalId = {}'.format(propId)
        nvisitBDict[metricName] = metricBundles.MetricBundle(nvisitMetric, \
                                nvisitSlicer, nvisitConstraint)

In [29]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
for metric in nvisitBDict:
    nvisitBDict[metric].setSummaryMetrics(summaryMetrics)

#### 3. Loop over all OpSims in dbDir and run MAF
While constructing a metricBundleGroup from a dictionary (the cell below), you will need to provide the path to a directory (`metricDataPath` in the cell below) where you would like to store the metric data (this is **DIFFERENT** than path to the metric data, `outDir`). To construct metricbundles for plotting and further analysis, this path will be needed.

In [ ]:
metricDataPath = '/home/idies/workspace/Storage/{}/\
persistent/MAFOutput/DDF/MetricData/'.format(your_username)

# coadd metric
for run in dbRuns[0:5]:
    # must set run name for each opSim to store metric data into
    # separate files
    for metric in coaddBDict:
        coaddBDict[metric].setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(coaddBDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

In [ ]:
# nvisit metric
for run in dbRuns[0:2]:
    # must set run name for each opSim to store metric data into
    # separate files
    for metric in nvisitBDict:
        nvisitBDict[metric].setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(nvisitBDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()